# Variables, Scopes and Devices

When you build your model, it is a good practice -- when not a true necessity -- to organize your variables in a reasonable way, both *logically* and *physically*. Before getting into the tutorial...



In [1]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(23)

## Give a variable a proper name and share them
A variable scope defines a variable namespace. It is easy to see how a name scope is prepended to the fully qualified name of a variable:

In [2]:
tf.reset_default_graph()
with tf.variable_scope('Scope') as scope:
    x = tf.Variable(23, name='x')
print(x.name)
print(x.op.name)

Scope/x:0
Scope/x


As you can see, the `tf.Variable.name` and the `tf.Variable.op.name` are different. As explained [here](http://stackoverflow.com/a/34729874/1861627), the `tf.Variable.name` property maps to the name of the mutable Tensor in which that variable is stored. *Tensor names are generated from the name of the operation that produces them and the index of the output to which that tensor corresponds* -- which is a variable operation in our case. To With a more *sophisticated* example:

In [3]:
tf.reset_default_graph()
with tf.variable_scope('Scope') as scope:
    a = tf.Variable(0, 'a')
    b = tf.Variable(0, 'b')
    c = tf.add(a, b, name='my_sum_op')
print(c.name)
print(c.op.name)

Scope/my_sum_op:0
Scope/my_sum_op


Even if straightforward, invoking directly the `tf.Variable` constructor to create a new variable **is not the more convenient way** to create your variables. The constructor will create a new variable each time and this is not what you want when you need to have shared variables. The proper way to overcome this issue is to use the `tf.get_variable()` function and properly set the scope `reuse` property.

In [4]:
tf.reset_default_graph()
with tf.variable_scope('ReuseScope') as scope:
    x1 = tf.get_variable(name='x', dtype=tf.int32, initializer=23)
    scope.reuse_variables()  # here!
    x2 = tf.get_variable(name='x', dtype=tf.int32, initializer=23)
    assert x1 == x2

Suppose we are in a recurrent scenario, when an input vector is fed into some layer: you want to *share the weights* of such layer across the different timesteps. This is achievable through such mechanism, as in the example below. Our `input` has 3 timesteps and each input value is a vector of 4 dimensions. We feed such sequence into a linear layer obtaining 3 outputs of 2 dimensions.

In [5]:
tf.reset_default_graph()
inputs = [
    tf.random_normal([1, 4]),
    tf.random_normal([1, 4]),
    tf.random_normal([1, 4]),
]
outputs = []

for t, inp in enumerate(inputs):
    with tf.variable_scope('RecurrentScope') as scope:
        if t > 0:
            scope.reuse_variables()
        weights = tf.get_variable(name='weights', shape=[4, 2], dtype=tf.float32)
        output = tf.matmul(inp, weights)
        outputs.append(output)

print('The variables in our model are:')
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print('   ' + var.op.name)

The variables in our model are:
   RecurrentScope/weights


> **NOTA BENE**: the example above is not the best way (if not the worst! :-)) to implement recurrencies and to represent inputs. Please, focus on what is inside the `for` loop and on how we can reuse already defined variables in order to share them across different operators.

What if we use the constructor? We will have *a new variable* created at each time which is *noT* what we expect if we want to reuse/share them.

In [6]:
tf.reset_default_graph()

for t in range(3):
    with tf.variable_scope('RecurrentScope') as scope:
        if t > 0:
            scope.reuse_variables()
        x = tf.Variable(23, name='x', dtype=tf.float32)

print('The variables in our model are:')
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print('   ' + var.op.name)

The variables in our model are:
   RecurrentScope/x
   RecurrentScope_1/x
   RecurrentScope_2/x


## Place the variable on the proper device
We can define a context also for the device we want the variable to be *physically* deploied on. The device name follow a certain convention which is extremely handy in a distributed training scenario. To the extent of this tutorial, it is enough to learn that you can address a device using its type, namely `CPU` or `GPU` and its index--an integer starting from 0 and ranging up to the amount of such devices (minus one, of course :-)).

In [7]:
tf.reset_default_graph()
with tf.variable_scope('GPUScope') as scope:
    with tf.device('GPU:999'):
        x = tf.constant(23, name='x', dtype=tf.float32)
        print('%s has been manually assigned to the device %s' % (x.op.name, x.device))

with tf.variable_scope('CPUScope') as scope:
    with tf.device('CPU:0'):
        y = tf.constant(9, name='y', dtype=tf.float32)
        print('%s has been manually assigned to the device %s' % (y.op.name, y.device))

GPUScope/x has been manually assigned to the device /device:GPU:999
CPUScope/y has been manually assigned to the device /device:CPU:0


At run-time[1], if the desired devices are note physically present, TF will raise an error. 

In [8]:
try:
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
except Exception as e:
    print e

Cannot assign a device to node 'GPUScope/x': Could not satisfy explicit device specification '/device:GPU:999' because no devices matching that specification are registered in this process; available devices: /job:localhost/replica:0/task:0/cpu:0
	 [[Node: GPUScope/x = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 23>, _device="/device:GPU:999"]()]]

Caused by op u'GPUScope/x', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/usr

You can ask it to softly place your variable and it will search for a fallback strategy. Typically, TF will try to allocate variables on `GPU:0`, falling back on the `CPU` if a `GPU` is not detected on the machine. If there is only one `GPU:0` available on the machine and you explicitly use a `GPU:1`, TF will fall back on the available devices.

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)
try:
    with tf.Session(config=tf.ConfigProto(
            log_device_placement=True,
            allow_soft_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(x))
        print(sess.run(y))
except Exception as e:
    print e

23.0
9.0


Unfortunately, Jupyter Notebooks don't get along with the TF logging system, but in the console you are running the kernel in, you should see the following:

```bash
2017-05-01 00:31:51.376794: I tensorflow/core/common_runtime/direct_session.cc:257] Device mapping:

init_10: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378591: I tensorflow/core/common_runtime/simple_placer.cc:841] init_10: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_9: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378649: I tensorflow/core/common_runtime/simple_placer.cc:841] init_9: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_8: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378683: I tensorflow/core/common_runtime/simple_placer.cc:841] init_8: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_7: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378715: I tensorflow/core/common_runtime/simple_placer.cc:841] init_7: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_6: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378747: I tensorflow/core/common_runtime/simple_placer.cc:841] init_6: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_5: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378779: I tensorflow/core/common_runtime/simple_placer.cc:841] init_5: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_4: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378810: I tensorflow/core/common_runtime/simple_placer.cc:841] init_4: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_3: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378841: I tensorflow/core/common_runtime/simple_placer.cc:841] init_3: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_2: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378872: I tensorflow/core/common_runtime/simple_placer.cc:841] init_2: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init_1: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378927: I tensorflow/core/common_runtime/simple_placer.cc:841] init_1: (NoOp)/job:localhost/replica:0/task:0/cpu:0
init: (NoOp): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378954: I tensorflow/core/common_runtime/simple_placer.cc:841] init: (NoOp)/job:localhost/replica:0/task:0/cpu:0
CPUScope/y: (Const): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.378985: I tensorflow/core/common_runtime/simple_placer.cc:841] CPUScope/y: (Const)/job:localhost/replica:0/task:0/cpu:0
GPUScope/x: (Const): /job:localhost/replica:0/task:0/cpu:0
2017-05-01 00:31:51.379016: I tensorflow/core/common_runtime/simple_placer.cc:841] GPUScope/x: (Const)/job:localhost/replica:0/task:0/cpu:0
^[[B^[[B[I 00:32:12.488 NotebookApp] Saving file at /02-VarsScopeDevs.ipynb

Device mapping: no known devices.
```

where we can see how `x` and `y` have been placed to some available devices:
```bash
...
CPUScope/y: (Const): /job:localhost/replica:0/task:0/cpu:0
GPUScope/x: (Const): /job:localhost/replica:0/task:0/cpu:0
...
```